In [3]:
!pip install keras-tuner --upgrade

# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras_tuner import Hyperband

#  Import and read the charity_data.csv.
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(["EIN", "NAME"], axis=1)

application_df.head()

# Determine the number of unique values in each column.
application_df.nunique()

# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
application_counts = application_df["APPLICATION_TYPE"].value_counts()

# Choose a cutoff value and create a list of application types to be replaced
application_types_to_replace = list(application_counts[application_counts < 500].index)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app, "Other")

# Check to make sure replacement was successful
application_df['APPLICATION_TYPE'].value_counts()

# Look at CLASSIFICATION value counts to identify and replace with "Other"
classification_counts = application_df["CLASSIFICATION"].value_counts()
classification_counts

# Choose a cutoff value and create a list of classifications to be replaced
classifications_to_replace = list(classification_counts[classification_counts < 1000].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls, "Other")

# Check to make sure replacement was successful
application_df['CLASSIFICATION'].value_counts()

# Convert categorical data to numeric with `pd.get_dummies`
numeric_df = pd.get_dummies(application_df)
numeric_df.head()

# Split our preprocessed data into our features and target arrays
y = numeric_df["IS_SUCCESSFUL"].values
X = numeric_df.drop(columns=["IS_SUCCESSFUL"]).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Define the model-building function for Keras Tuner
def build_model(hp):
    model = keras.Sequential()

    # Input layer
    number_input_features = len(X_train_scaled[0])

    # Tune the number of hidden units in the first layer
    hp_units1 = hp.Int('units1', min_value=32, max_value=128, step=16)
    model.add(keras.layers.Dense(units=hp_units1, input_dim=number_input_features, activation='relu'))

    # Tune the number of hidden units in the second layer
    hp_units2 = hp.Int('units2', min_value=16, max_value=64, step=8)
    model.add(keras.layers.Dense(units=hp_units2, activation='relu'))

    # Output layer
    model.add(keras.layers.Dense(units=1, activation='sigmoid'))

    # Compile the model
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    return model

# Initialize the tuner
tuner = Hyperband(
    build_model,
    objective='val_accuracy',
    max_epochs=20,
    factor=3,
    directory='my_dir',
    project_name='charity_model_tuning'
)

# Perform hyperparameter search
tuner.search(X_train_scaled, y_train, epochs=20, validation_data=(X_test_scaled, y_test))

# Get the best hyperparameters and model
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

# Print the optimal hyperparameters
print(f"Optimal units for the first layer: {best_hps.get('units1')}")
print(f"Optimal units for the second layer: {best_hps.get('units2')}")

# Build the model with the best hyperparameters
model = tuner.hypermodel.build(best_hps)

# Train the model
history = model.fit(X_train_scaled, y_train, epochs=100, validation_data=(X_test_scaled, y_test))

# Evaluate the model
model_loss, model_accuracy = model.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

# Save the model
model.save('AlphabetSoupCharity_keras_tuner.h5')


Trial 30 Complete [00h 00m 46s]
val_accuracy: 0.7276967763900757

Best val_accuracy So Far: 0.728863000869751
Total elapsed time: 00h 09m 40s
Optimal units for the first layer: 96
Optimal units for the second layer: 48
Epoch 1/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.7022 - loss: 0.5922 - val_accuracy: 0.7270 - val_loss: 0.5618
Epoch 2/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7277 - loss: 0.5590 - val_accuracy: 0.7184 - val_loss: 0.5576
Epoch 3/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7327 - loss: 0.5511 - val_accuracy: 0.7210 - val_loss: 0.5581
Epoch 4/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7358 - loss: 0.5499 - val_accuracy: 0.7271 - val_loss: 0.5529
Epoch 5/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7334 - loss: 0.5469 - val_accuracy: 0.7273 - val_loss: 0.5545
Epoch 6/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.7315 - loss: 0.5500 - val_accuracy: 0.7275 - val_loss: 0.5538
Epoch 7

Loss: 0.5571195483207703, Accuracy: 0.7268804907798767
